In [3]:
from numpy import where
from pandas import read_csv
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM


In [4]:
all_data = read_csv("../data/PAMAP2/OAO_data.csv")
all_data.shape

(175498, 55)

In [ ]:
# os ultimos 20% das atividades de todos os subjects
# achar um conjunto otimo para todas as novidades.
# metricas de interação com os hyperparametros
# 2º quadro com os melhores hyperparams, dados as metricas

In [5]:
for activity in all_data["activity"]:
    novelty = all_data[all_data["activity"] == activity]
    current_data = all_data[~novelty.index]

	# Target
    current_data["isNovelty"] = False
    novelty["isNovelty"] = True

    X_train = current_data.drop(columns=["activity", "isNovelty"])
    y_train = current_data["isNovelty"]
    
	# only current activity (as novelty)
    X_test = novelty.drop(columns=["activity", "isNovelty"])
    y_test = novelty["isNovelty"]

    oc_svm = OneClassSVM(kernel="rbf").fit(X_train)
    y_pred = oc_svm.predict(X_test)
    y_pred = where(y_pred == 1, False, True)

    print(f"Activity: {activity}, F1 Score: {f1_score(y_test, y_pred, pos_label=True)}")

KeyError: "None of [Index([     -1,      -2,      -3,      -4,      -5,      -6,      -7,      -8,\n            -9,     -10,\n       ...\n       -153492, -153493, -153494, -153495, -153496, -153497, -153498, -153499,\n       -153500, -153501],\n      dtype='int64', length=17575)] are in the [columns]"